In [1]:
# import SQL Alchemy
from sqlalchemy import create_engine

# Import and establesh BAse for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data type
from sqlalchemy import Column, Integer, String

# Import Session to push objects made and query the server.
from sqlalchemy.orm import Session

import pandas as pd

In [2]:
# Create Accident Class
class Accident(Base):
    __tablename__ = "accidents"
    id = Column(Integer,primary_key=True)
    case_num = Column(Integer)
    veh_num = Column(Integer)
    region = Column(String)
    region_num = Column(Integer)
    month = Column(String)
    hour = Column(String)
    make = Column(String)
    passenger_inj = Column(String)
    num_injured = Column(Integer)
    alcohol_involved = Column(String)
    situation = Column(String,)
    road_condition = Column(String)
    veh_damage = Column(String)

In [3]:
# Create and establish Database Connection
engine = create_engine("sqlite:///accidents.sqlite")
conn = engine.connect()

# Create Accident table within the database
Base.metadata.create_all(conn)

In [4]:
# Create Session to push the objects made and query the server.
session = Session(bind=engine)

# Readig csv file

In [5]:
import pandas as pd
path = "Resources/veh_19.csv"
data = pd.read_csv(path)
data_df = pd.DataFrame(data)
data_df.head()

,CASENUM,VEH_NO,REGIONNAME,REGION,MONTHNAME,HOURNAME,MAKENAME,MXVSEV_IMNAME,NUMINJ_IM,V_ALCH_IMNAME,PCRASH1_IMNAME,VSURCONDNAME,DEFORMEDNAME
0,2.019012e+11,1,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",3,January,8:00am-8:59am,Ford,No Apparent Injury (O),0,No Alcohol Involved,Accelerating in Road,Wet,Disabling Damage
1,2.019012e+11,1,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",3,January,2:00am-2:59am,Dodge,Possible Injury (C),1,No Alcohol Involved,Going Straight,Dry,Functional Damage
2,2.019012e+11,2,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",3,January,2:00am-2:59am,Honda,Possible Injury (C),1,No Alcohol Involved,Going Straight,Dry,Minor Damage
3,2.019012e+11,1,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",3,January,5:00am-5:59am,Nissan/Datsun,Suspected Minor Injury (B),1,Alcohol Involved,Changing Lanes,Dry,Disabling Damage
4,2.019012e+11,2,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",3,January,5:00am-5:59am,Chevrolet,Possible Injury (C),1,No Alcohol Involved,Turning Left,Non-Trafficway or Driveway Access,Disabling Damage


In [10]:
regions = data_df['REGIONNAME'].unique()
regions

array(['South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA, FL, AL, MS, LA, AR, OK, TX)',
       'West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, WY, AK, HI)',
       'Midwest (OH, IN, IL, MI, WI, MN, ND, SD, NE, IA, MO, KS)',
       'Northeast (PA, NJ, NY, NH, VT, RI, MA, ME, CT)'], dtype=object)

In [14]:
# Clean RegionName of the list of states for each region
region_name = data_df['REGIONNAME']
for i in range(len(data_df['REGIONNAME'])):
    row = data_df['REGIONNAME'][i]
    if row == regions[0]:
        region_name[i] = "South"
    elif row == regions[1]:
        region_name[i] = "West"
    elif row == regions[2]:
        region_name[i] = "Mid West"
    elif row == regions[3]:
        region_name[i] = "North East"
region_name.unique()

array(['South', 'West', 'Mid West', 'North East'], dtype=object)

In [16]:
# Assing the region_name to the correct column in the DataFrame
data_df['REGIONNAME'] = region_name
data_df['REGIONNAME'].unique()

array(['South', 'West', 'Mid West', 'North East'], dtype=object)

In [19]:
data_df.dtypes

CASENUM           float64
VEH_NO              int64
REGIONNAME         object
REGION              int64
MONTHNAME          object
HOURNAME           object
MAKENAME           object
MXVSEV_IMNAME      object
NUMINJ_IM           int64
V_ALCH_IMNAME      object
PCRASH1_IMNAME     object
VSURCONDNAME       object
DEFORMEDNAME       object
dtype: object

In [24]:
data_df = data_df.astype({"CASENUM":"int64"})

In [25]:
data_df.dtypes

CASENUM            int64
VEH_NO             int64
REGIONNAME        object
REGION             int64
MONTHNAME         object
HOURNAME          object
MAKENAME          object
MXVSEV_IMNAME     object
NUMINJ_IM          int64
V_ALCH_IMNAME     object
PCRASH1_IMNAME    object
VSURCONDNAME      object
DEFORMEDNAME      object
dtype: object

# From DataFrame to Database

In [26]:
for i in range(len(data_df.index)):
    
    # Create instance of the accident
    accident = Accident(
    id = i,
    case_num = data_df['CASENUM'][i],
    veh_num = data_df['VEH_NO'][i],
    region = data_df['REGIONNAME'][i],
    region_num = data_df['REGION'][i],
    month = data_df['MONTHNAME'][i],
    hour = data_df['HOURNAME'][i],
    make = data_df['MAKENAME'][i],
    passenger_inj = data_df['MXVSEV_IMNAME'][i],
    num_injured = data_df['NUMINJ_IM'][i],
    alcohol_involved = data_df['V_ALCH_IMNAME'][i],
    situation = data_df['PCRASH1_IMNAME'][i],
    road_condition = data_df['VSURCONDNAME'][i],
    veh_damage = data_df['DEFORMEDNAME'][i])
    
    # Add accident 'i' to the Database
    session.add(accident)
    # commit object to the Database
    session.commit()

In [28]:
data_df["CASENUM"][2]

201901176655

In [46]:
for row in session.query(Accident).limit(2).all():
    print(row)

In [38]:
print(accdt)

SELECT accidents.id AS accidents_id, accidents.case_num AS accidents_case_num, accidents.veh_num AS accidents_veh_num, accidents.region AS accidents_region, accidents.region_num AS accidents_region_num, accidents.month AS accidents_month, accidents.hour AS accidents_hour, accidents.make AS accidents_make, accidents.passenger_inj AS accidents_passenger_inj, accidents.num_injured AS accidents_num_injured, accidents.alcohol_involved AS accidents_alcohol_involved, accidents.situation AS accidents_situation, accidents.road_condition AS accidents_road_condition, accidents.veh_damage AS accidents_veh_damage 
FROM accidents 
WHERE accidents.case_num = ?
